In [ ]:
!pip install transformers datasets accelerate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from datasets import load_dataset

In [ ]:
ds_small_clean = load_dataset("iTzMiNOS/voice-orders-small-clean-12k")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
ds_small_clean

DatasetDict({
    train: Dataset({
        features: ['transcribed_text', 'speaker', 'items', 'count', 'audio'],
        num_rows: 12000
    })
})

In [ ]:
ds_small_clean_af_bella = ds_small_clean.filter(lambda example: example['speaker'] == 'af_bella')

ds_small_clean_noaudio = ds_small_clean_af_bella.select_columns(["transcribed_text", "speaker", "items", "count"])

In [ ]:
ds_small_clean_noaudio

DatasetDict({
    train: Dataset({
        features: ['transcribed_text', 'speaker', 'items', 'count'],
        num_rows: 1200
    })
})

In [ ]:
def addText(example):
    return f"translate to json: {example}"
ds_small_clean_noaudio['train'] = ds_small_clean_noaudio['train'].map(lambda example: {'input_text': addText(example['transcribed_text'])})

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [ ]:
ds_small_clean_noaudio['train'][0]['input_text']

"translate to json: I'll have a Cream of Mushroom with truffle oil and chopped parsley, a Greek salad without protein, add grilled chicken and falafel, a vegan burger with mustard, ketchup, no cheese, vegan cheese, and vegan mayo, plus extra cheese on the side."

In [ ]:
ds_small_clean_noaudio['train'][0]['items']

[{'customizations': ['Truffle Oil', 'Chopped Parsley'],
  'name': 'Cream of Mushroom'},
 {'customizations': ['No Protein', 'Grilled Chicken', 'Falafel'],
  'name': 'Greek Salad'},
 {'customizations': ['Mustard',
   'Ketchup',
   'No Cheese',
   'Vegan Cheese',
   'Vegan Mayo'],
  'name': 'Vegan Burger'},
 {'customizations': [], 'name': 'Extra Cheese'}]

In [ ]:
import json
def stringify_items(example):
    return {'target_text': json.dumps(example['items'], indent=0)}
ds_small_clean_noaudio['train'] = ds_small_clean_noaudio['train'].map(lambda example: stringify_items(example))

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [ ]:
ds_small_clean_noaudio['train'][0]['target_text']

'[\n{\n"customizations": [\n"Truffle Oil",\n"Chopped Parsley"\n],\n"name": "Cream of Mushroom"\n},\n{\n"customizations": [\n"No Protein",\n"Grilled Chicken",\n"Falafel"\n],\n"name": "Greek Salad"\n},\n{\n"customizations": [\n"Mustard",\n"Ketchup",\n"No Cheese",\n"Vegan Cheese",\n"Vegan Mayo"\n],\n"name": "Vegan Burger"\n},\n{\n"customizations": [],\n"name": "Extra Cheese"\n}\n]'

In [ ]:
from datasets import Dataset, DatasetDict

split_dataset = ds_small_clean_noaudio['train'].train_test_split(test_size=0.1)

In [ ]:
ds_small_clean_processed = split_dataset.select_columns(["input_text", "target_text"])

In [ ]:
ds_small_clean_processed

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 1080
    })
    test: Dataset({
        features: ['input_text', 'target_text'],
        num_rows: 120
    })
})

In [1]:
from transformers import T5Tokenizer
model_name = "iTzMiNOS/t5-base-2-4-ipo-txtonly"
tokenizer = T5Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    inputs = examples['input_text']
    targets = examples['target_text']

    # Tokenize inputs and targets with padding and truncation
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length", return_tensors="pt")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length", return_tensors="pt")["input_ids"]

    # Ensure labels are correctly formatted
    model_inputs["labels"] = labels
    return model_inputs
tokenized_dataset = ds_small_clean_processed.map(preprocess_function, batched=True)

Map:   0%|          | 0/1080 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1080
    })
    test: Dataset({
        features: ['input_text', 'target_text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 120
    })
})

In [ ]:
from transformers import EvalPrediction
import numpy as np

def compute_metrics(eval_pred: EvalPrediction):
    # Unpack the predictions and labels
    predictions, labels = eval_pred

    # Ensure predictions and labels have the correct shape
    predictions = np.argmax(predictions, axis=-1)  # Get token IDs from logits (logits -> IDs)

    # Make sure both predictions and labels are 2D (batch_size, sequence_length)
    predictions = predictions.reshape(-1)  # Flatten if necessary
    labels = labels.reshape(-1)  # Flatten if necessary

    # Replace -100 with pad_token_id in labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decode predictions and labels into strings
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    correct_predictions = 0
    for pred, label in zip(decoded_preds, decoded_labels):
        if pred.strip() == label.strip():
            correct_predictions += 1

    accuracy = correct_predictions / len(decoded_preds)
    return {"accuracy": accuracy}


In [ ]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer
model = T5ForConditionalGeneration.from_pretrained(model_name)

''' training_args = TrainingArguments(
    output_dir="./flan-t5-base-finetuned-json-small",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_steps=50,
    push_to_hub=False
) '''

training_args = TrainingArguments(
    output_dir="./flan-t5-base-finetuned-json-small",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_steps=50,
    push_to_hub=False,
    fp16=True,
    report_to=None,
    fp16_full_eval=True,
    dataloader_num_workers=4,
    load_best_model_at_end=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(
  tokenizer=tokenizer,
  model=model
)

In [ ]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_name)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,  # tokenizer can be replaced by processing_class
    data_collator=data_collator
)
trainer.train()

<ipython-input-97-3d6a3d43a97c>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.000000,nan
2,0.000000,nan
3,0.000000,nan


TrainOutput(global_step=204, training_loss=0.0, metrics={'train_runtime': 58.8772, 'train_samples_per_second': 55.03, 'train_steps_per_second': 3.465, 'total_flos': 554653455482880.0, 'train_loss': 0.0, 'epoch': 3.0})

In [5]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
def predict_json(order_text):
    input_text = f"translate to json: {order_text}"

    device = next(model.parameters()).device

    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

    outputs = model.generate(input_ids, max_length=512)

    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return prediction

sample_text = "I'd like a vegan burger with no cheese and extra pickles, a chicken wrap with spicy mayo, a caesar salad without croutons, a beef taco with extra salsa and guacamole, a margherita pizza with extra basil, a tuna sandwich with no onions, a greek salad with olives and feta, a falafel plate with hummus and tahini, fries with ketchup and no salt, and finally, a chocolate milkshake with almond milk."
predicted_json = predict_json(sample_text)
print("Predicted JSON:", predicted_json)

Predicted JSON: [  "customizations": [ "No Cheese", "Extra Pickles" ], "name": "Vegan Burger" ,  "customizations": [ "Spicy Mayo" ], "name": "Chicken Wrap" ,  "customizations": [ "Croutons" ], "name": "Caesar Salad" ,  "customizations": [ "Extra Salsa", "Extra Basil" ], "name": "Margherita" ,  "customizations": [ "Olives", "Feta" ], "name": "Feta" ,  "customizations": [ "Almond Milk" ], "name": "Chocolate Milkshake"  ]


In [ ]:
sample_text = "I'd like a Spaghetti Carbonara with extra parmesan, a Veggie Burger with avocado, and a Thai Green Curry with mild spice."
predicted_json = predict_json(sample_text)
print("Predicted JSON:", predicted_json)

Predicted JSON: Json a Spaghetti Carbonara w ess essip parmesan, a Veggie Burger w avocado, a Thai Green Curry w essip smoky.


In [ ]:
sample_text = "Can I get a Chicken Teriyaki with extra sauce, a Margherita Pizza with gluten-free crust, and a Caesar Salad with no croutons?"
predicted_json = predict_json(sample_text)
print("Predicted JSON:", predicted_json)

Predicted JSON: 


In [ ]:
trainer.save_model("t5-large-json-order-finetuned")

In [ ]:
import zipfile
import os

def zip_folder(folder_path, output_zip):
    # Create a ZipFile object in write mode
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk through the folder and add all files to the zip file
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_path))

# Example usage
folder_path = '/content/t5-large-json-order-finetuned'  # replace with your folder path
output_zip = 't5-large-json-order-finetuned.zip'  # replace with your desired output zip file name
zip_folder(folder_path, output_zip)

print(f"Folder '{folder_path}' has been zipped as '{output_zip}'.")

Folder '/content/t5-large-json-order-finetuned' has been zipped as 't5-large-json-order-finetuned.zip'.
